# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [44]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math 
import warnings
warnings.filterwarnings('ignore')

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available in sp_500_stocks.csv.

In [120]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.shape
stocks.head(10)

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
5,ABC
6,ABMD
7,ABT
8,ACN
9,ADBE


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

In [3]:
from tokens import IEX_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock

In [6]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/stock/{symbol}/quote/?token={IEX_TOKEN}'
data = requests.get(api_url, verify=False).json()
# print(data.status_code)

/Users/saeed/Projects/algotrading/training/youtube_toturial_1/Equal_weight_SP500/env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [17]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [37]:
column_name = ['ticker', 'stock_price', 'market_cap', 'number_of_shares_to_buy']
df = pd.DataFrame(columns=column_name)
df = pd.concat([
    df,
    pd.DataFrame([[
        symbol,
        price,
        market_cap,
        'N/A'
    ]], columns=column_name)
], ignore_index=True)
df

/var/folders/hg/dj7n4q9n539g0kq8zl0tcvvw0000gn/T/ipykernel_10350/2649977992.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [46]:
df = pd.DataFrame(columns=column_name)
for stock in stocks['Ticker'][:5]:
    api_url  = f'https://api.iex.cloud/v1/stock/{stock}/quote/?token={IEX_TOKEN}'
    data = requests.get(api_url, verify=False).json()
    df = pd.concat([
        df,
        pd.DataFrame([[
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ]], columns=column_name)
    ], ignore_index=True)

In [47]:
df

,ticker,stock_price,market_cap,number_of_shares_to_buy
0,A,131.16,38430418018,N/A
1,AAL,12.95,8463350123,N/A
2,AAP,61.00,3629621390,N/A
3,AAPL,184.25,2865594556000,N/A
4,ABBV,160.46,288828000000,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [75]:
from typing import Type
def spliter(ser:Type[pd.Series], n:int ) -> list:
    if np.mod(ser.size,n) == 0:
        number_of_groups = int(ser.size/n)
    else:
        number_of_groups = int(ser.size/n) + 1
    l2 = []
    for i in range(number_of_groups):
        l1 = []
        for j in range(i*n,min((i+1)*n, ser.size)):
            l1.append(ser.iloc[j])
        l2.append(l1)
    return l2

100

In [144]:
splitted_stock = spliter(stocks.Ticker,100)
df = pd.DataFrame(columns=column_name)
for i,symbol_lst in enumerate(splitted_stock): 
    string_stocks = ','.join(symbol_lst)
    batch_api_url = f'https://api.iex.cloud/v1/stock/market/batch?symbols={string_stocks}&types=quote&token={IEX_TOKEN}'
    data = requests.get(batch_api_url, verify=False).json()
    print(i)
    for symbol in symbol_lst:
        symbol_data = data.get(symbol,None)
        if symbol_data:
            price = symbol_data['quote']['latestPrice']
            market_cap = symbol_data['quote']['marketCap']
        else:
            price = 'N/A'
            market_cap = 'N/A'
        df_extend = pd.DataFrame([[symbol, price, market_cap, 'N/A']], columns=column_name)
        df = pd.concat([df, df_extend], ignore_index=True)
    print(i)
df

0
0
1
1
2
2
3
3
4
4
5
5


,ticker,stock_price,market_cap,number_of_shares_to_buy
0,A,130.98,38377677280,N/A
1,AAL,13.115,8571184313,N/A
2,AAP,61.08,3634381549,N/A
3,AAPL,181.185,2817925371120,N/A
4,ABBV,161.37,290466000000,N/A
...,...,...,...,...
500,YUM,129.18,36210215730,N/A
501,ZBH,120.41,25163367412,N/A
502,ZBRA,252.02,12943647400,N/A
503,ZION,42.8,6340768597,N/A


In [165]:
# dropping null rows
df.replace('N/A', np.nan, inplace=True)
df = df[~df['stock_price'].isnull()]
len(df)

496

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [167]:
while True:
    portfolio_input = input('Enter the value of your portfolio: ')
    try:
        portfolio_size = float(portfolio_input)
        break
    except ValueError:
        print('Enter an integer value! \nPlease Try again')

In [174]:
position_size = portfolio_size / len(df)
df['number_of_shares_to_buy'] = position_size / df['stock_price']
df['number_of_shares_to_buy'] = df['number_of_shares_to_buy'].apply(math.floor)
df.head()

,ticker,stock_price,market_cap,number_of_shares_to_buy
0,A,130.980,3.837768e+10,153
1,AAL,13.115,8.571184e+09,1537
2,AAP,61.080,3.634382e+09,330
3,AAPL,181.185,2.817925e+12,111
4,ABBV,161.370,2.904660e+11,124


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [175]:
df.to_excel('Trade Recommendation.xlsx', index=False)